This notebook will take a dataset of images, run them through TSNE to group them up (if enabled) then create a stylegan2 model with or without ADA.

Below are setting to choose when running this workflow. Make sure before running to have all images you want to use in a folder inside of the images folder. For example have a folder inside images called mona-lisa filled with pictures of different versions of the Mona Lisa. Please have the subfolder have no whitespaces in the name.

If TSNE is enable the program will halt after processing the images and ask you to choose which cluster to use. The clusters will be in the folder clusters.

Before running make sure your kernal is set to Python 3 (TensorFlow 1.15 Python 3.7 GPU Optimized)

In [1]:
dataset_name = 'mona-lisa'

use_ada = True
use_tsne = False
use_spacewalk = True


# Crop Settings
# Choose center or no-crop
# TODO: Add random
crop_type = 'no-crop'
resolution = 512


# TSNE Settings
# Choose number of clusters to make or None for auto clustering
num_clusters = None


# ADA Settings
knum = 10


# Spacewalk Settings
fps = 24
seconds = 10
#Leave seeds = None for random seeds or 
# enter a list in the form of [int, int, int..] to define the seeds
seeds = None
# set walk_type to 'line', 'sphere', 'noiseloop', or 'circularloop'
walk_type = 'sphere'


In [2]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import os
import train
from PIL import Image, ImageFile, ImageOps
import shutil
import math

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
def resize(pil_img, res):
    return pil_img.resize((res, res))

def crop_center(pil_img, res):
    
    crop = res
    
    img_width, img_height = pil_img.size
    if img_width < crop:
        crop = img_width
    if img_height < crop:
        crop = img_height
        
    a = (img_width - crop) // 2
    b = (img_height - crop) // 2
    c = (img_width + crop) // 2
    d = (img_height + crop) // 2
        
    cropped_image = pil_img.crop((a,b,c,d))
    return resize(cropped_image, res)

def no_crop(pil_img, res):
    color = [0, 0, 0]
    
    img_width, img_height = pil_img.size
    if img_width < img_height:
        top = 0
        bottom = 0
        left =  math.ceil((img_height - img_width) / 2.0)
        right =  math.floor((img_height - img_width) / 2.0)
    else:
        top =  math.ceil((img_height - img_width) / 2.0)
        bottom =  math.floor((img_height - img_width) / 2.0)
        left = 0
        right = 0
    
    border_image = ImageOps.expand(pil_img, border=(left, top, right, bottom), fill='white')
    return resize(border_image, res)

In [5]:
image_dir = './images/'
tmp_dir = './tmp/'

image_dir = os.path.join(image_dir, dataset_name)
tmp_dir = os.path.join(tmp_dir, dataset_name)


if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
else:
    try:
        shutil.rmtree(tmp_dir)
    except OSError as e:
        print("Error: %s : %s" % (dir_path, e.strerror))
    os.makedirs(tmp_dir)
    

for filename in os.listdir(image_dir):
    file_extension = os.path.splitext(filename)[-1]
    if file_extension != '.jpg' and file_extension != '.png':
        print(file_extension)
        continue
    
    image_path = os.path.join(image_dir, filename)
    image = Image.open(image_path)
    mode = image.mode
    if str(mode) != 'RGB':
        continue
    if crop_type == "center":
        image = crop_center(image, resolution)
    if crop_type == "no-crop":
        image = no_crop(image, resolution)
        
    tmp_path = os.path.join(tmp_dir, filename)
    image.save(tmp_path)

In [6]:
if use_tsne:
    !python tsne.py --path={tmp_dir}
else:
    print('TSNE is not in use')

TSNE is not in use


If TSNE is enabled when it is finished running check the Clusters folder and choose the cluster you want to use below

In [7]:
if use_tsne:
    clusters = []
    while True:
        x = input("Enter a cluster you want to use or Enter to continue: ")
        if x == '':
            break
        clusters.append(int(x))

In [8]:
if use_tsne:
    save_dir = os.path.join("./tmp", str(dataset_name + "_clusters"))
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    else:
        try:
            shutil.rmtree(save_dir)
        except OSError as e:
            print("Error: %s : %s" % (save_dir, e.strerror))
        os.makedirs(save_dir)
                            
    for c in clusters:
        cluster_dir = os.path.join("./clusters", dataset_name, str("cluster_" + str(c)))
        
        for filename in os.listdir(cluster_dir):
            file = os.path.join(cluster_dir, filename)
            new_file = os.path.join(save_dir, filename)
            shutil.move(file, new_file)
    

In [ ]:
dataset_dir = os.path.join("./datasets", dataset_name)

if use_ada and use_tsne:
    image_dir = os.path.join("./tmp", str(dataset_name + "_clusters"))
    !python dataset_tool.py create_from_images {dataset_dir} {image_dir}
    !python train.py --outdir=./training-runs --gpus=4 --res={resolution} --data={dataset_dir} --kimg={knum}
elif use_ada:
    image_dir = os.path.join("./tmp", dataset_name)
    !python dataset_tool.py create_from_images {dataset_dir} {image_dir}
    !python train.py --outdir=./training-runs --gpus=4 --res={resolution} --data={dataset_dir} --kimg={knum}
else:
    print("ADA is not in use")
    


Loading images from "./tmp/mona-lisa"
Creating dataset "./datasets/mona-lisa"
Added 529 images.                       


Training options:
{
  "G_args": {
    "func_name": "training.networks.G_main",
    "fmap_base": 16384,
    "fmap_max": 512,
    "mapping_layers": 2,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "D_args": {
    "func_name": "training.networks.D_main",
    "mbstd_group_size": 4,
    "fmap_base": 16384,
    "fmap_max": 512,
    "num_fp16_res": 4,
    "conv_clamp": 256
  },
  "G_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "D_opt_args": {
    "beta1": 0.0,
    "beta2": 0.99,
    "learning_rate": 0.0025
  },
  "loss_args": {
    "func_name": "training.loss.stylegan2",
    "r1_gamma": 1.6384
  },
  "augment_args": {
    "class_name": "training.augment.AdaptiveAugment",
    "tune_heuristic": "rt",
    "tune_target": 0.6,
    "apply_func": "training.augment.augment_pipeline",
    "apply_args": {
      "xflip": 1,
      "rotate9